In [2]:

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Mar 21 00:06:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
#install aimodelshare library
! pip install aimodelshare --upgrade
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 967.8/967.8 KB 28.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.3/442.3 KB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 101.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 KB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.4/178.4 KB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.0/294.0 KB 34.4 MB/s eta 0:00:00
     ━━━━━━

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as functional
import matplotlib.pyplot as plt
from transformers import BertForSequenceClassification, AdamW, BertConfig
import gc
from transformers import BertModel
from sklearn.metrics import roc_auc_score,f1_score
import time
import datetime

In [5]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [6]:
torch.manual_seed(0)
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
if use_cuda:
    torch.cuda.manual_seed(0)
    
print("Using GPU: {}".format(use_cuda))

Using GPU: True


In [7]:
# Read the rumor dataset to train the model parameters
df = pd.read_csv('/content/drive/My Drive/Misinformation_Project/rumor_dataset.csv') 
df["Veracity"] = df.Veracity.map({"F":0, "T":1, "U":0}) 
df.replace([np.inf, -np.inf], 2) 
df.fillna(2, inplace=True)
df.head()

,Index,Veracity,Text,Sentiment
0,3,0.0,The lie that coronavirus came from a bat or a ...,3
1,4,0.0,The health experts had predicted the virus cou...,3
2,8,0.0,The Centers for Disease Control and Prevention...,3
3,10,0.0,Warm weather will kill coronavirus. U.S. Presi...,2
4,15,0.0,Using a hair dryer to breathe in hot air can c...,2


In [8]:
# Split into traning and testing sets
from sklearn.model_selection import train_test_split 

train, test = train_test_split(df, test_size=0.2)

In [9]:
# Import libraries for the model training
import tensorflow as tf
import re, string, unicodedata
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

#BERT
import transformers
from transformers import TFBertModel
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

In [15]:
def bert_encode(data, maximum_length) :
    input_ids = []
    attention_masks = []

    for text in data:
        encoded = tokenizer.encode_plus(
            text, 
            add_special_tokens=True,
            max_length=maximum_length,
            pad_to_max_length=True,

            return_attention_mask=True,
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        
    return np.array(input_ids),np.array(attention_masks)

In [16]:
def create_model(bert_model):
    
    input_ids = tf.keras.Input(shape=(310,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(310,),dtype='int32')

    output = bert_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(64,activation='relu')(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(1,activation='sigmoid')(output)
    
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    return model
     

In [17]:
#load pre-trained uncased BERT
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [18]:
model_BERT_loss1e5_epoch5 = create_model(bert_model)
model_BERT_loss1e5_epoch5.summary

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


<bound method Model.summary of <keras.engine.functional.Functional object at 0x7fa6dd634eb0>>

In [19]:
train_input_ids, train_attention_masks = bert_encode(train["Text"], 310)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [21]:
history = model_BERT_loss1e5_epoch5.fit(
    [train_input_ids, train_attention_masks],
    train["Veracity"],
    epochs=5,
    batch_size=16
)

Epoch 1/5
308/308 [==============================] - 315s 1s/step - loss: 0.4219 - accuracy: 0.8150
Epoch 2/5
308/308 [==============================] - 315s 1s/step - loss: 0.3160 - accuracy: 0.8614
Epoch 3/5
308/308 [==============================] - 314s 1s/step - loss: 0.2242 - accuracy: 0.9030
Epoch 4/5
308/308 [==============================] - 314s 1s/step - loss: 0.1458 - accuracy: 0.9398
Epoch 5/5
308/308 [==============================] - 314s 1s/step - loss: 0.0851 - accuracy: 0.9671


In [22]:
test_input_ids, test_attention_masks = bert_encode(test["Text"], 310)

In [23]:
predictions = model_BERT_loss1e5_epoch5.predict((test_input_ids, test_attention_masks))

39/39 [==============================] - 30s 700ms/step


In [24]:
prediction_labels = [1 if x > 0.5 else 0 for x in predictions]
prediction_labels_df = pd.DataFrame({"labels":prediction_labels,"tweet":test["Text"]})

In [25]:
prediction_labels_df.to_csv("/content/drive/MyDrive/Colab Notebooks/bert_2.csv")

In [26]:
!pip install -U tf2onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
import tf2onnx
import onnx

model_proto, external_tensor_storage = tf2onnx.convert.from_keras(model_BERT_loss1e5_epoch5)
onnx.save(model_proto, "/content/drive/MyDrive/Colab Notebooks/final_model.onnx")
model_BERT_loss1e5_epoch5.save_weights("/content/drive/MyDrive/Colab Notebooks/bert_weights")
    

In [29]:
# read our own vaccine-related Twitter data
df_vaccine = pd.read_csv('/content/drive/My Drive/Misinformation_Project/21-22 two year updated dataset/individualinf_repsonseeff_30K_vaccine_21_22.csv') 

In [30]:
vaccine_input_ids, vaccine_attention_masks = bert_encode(df_vaccine["Processed Tweets"], 310)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [31]:
vaccine_predictions = model_BERT_loss1e5_epoch5.predict((vaccine_input_ids, vaccine_attention_masks))

658/658 [==============================] - 477s 725ms/step


In [32]:
df_vaccine["labels"] = [1 if x > 0.5 else 0 for x in vaccine_predictions]
df_vaccine["Classify"]=df_vaccine.labels.map({0:"M", 1:"C"}) 

In [34]:
df_vaccine.head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,User,Date Created,Tweet,Likes,Followers,Friends,...,Polarity,Subjectivity,Individual Influence,Like Effect,Reply Effect,Retweet Effect,Response Effect,Subjective Level,labels,Classify
0,0,0,0,0,Treep52,2022-12-30 22:51:15+00:00,Per Alex: Approximately 95% of adults and 98% ...,1,3425,3949,...,0.100000,0.300000,3,1,0,0,0.333333,2,0,M
1,1,1,1,1,DrGurdeepParhar,2022-12-30 21:10:00+00:00,Big Tobacco's divestment from Quebec's Medicag...,0,66607,48,...,0.142857,0.317857,4,0,0,0,0.000000,2,0,M
2,2,2,2,2,stopthegaslight,2022-12-30 18:49:59+00:00,@brianlilley COVID Vaccinations are the most u...,0,17,189,...,0.350000,0.575000,1,0,0,0,0.000000,3,0,M
3,3,3,3,3,Daisy381,2022-12-30 17:31:01+00:00,@MerlinofCanada @JustinTrudeau In Alberta it’s...,1,331,598,...,0.516667,0.916667,2,1,1,0,0.666667,4,0,M
4,4,4,4,4,RadarsScience,2022-12-30 16:20:06+00:00,Big Tobacco&amp;#039;s divestment from Quebec&...,0,29,45,...,0.142857,0.317857,1,0,0,0,0.000000,2,1,C


In [35]:
# Function to determine sign of impact based on polarity of sentiment 
# We assume negative opinion on vaccine has a negative impact
def signofimpact(score):
    if score > 0:
        return 1
    else:
        return -1

# Function to check if the tweet contains hashtags that are hot topics
# Have a hashtag under a hot topic will increase visibility of the tweet
def ifhottopic(hashtags):
    match = ['COVID' , 'vaccine','covid','Toronto','Ottawa','cdnpoli','Covid','onpoli','Canada','ottnews']
    if any(x in hashtags for x in match):
        return 1
    else:
        return 0

In [36]:
df_vaccine["Sign of Impact"] = df_vaccine["Polarity"].apply(signofimpact)
df_vaccine["Impact Value"] = df_vaccine["Individual Influence"]+df_vaccine["Response Effect"]+df_vaccine["Subjectivity"]+df_vaccine["Hashtags"].astype(str).apply(ifhottopic)
df_vaccine["Overall Impact"] = df_vaccine["Sign of Impact"]*df_vaccine["Impact Value"]
df_vaccine.head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,User,Date Created,Tweet,Likes,Followers,Friends,...,Like Effect,Reply Effect,Retweet Effect,Response Effect,Subjective Level,labels,Classify,Sign of Impact,Impact Value,Overall Impact
0,0,0,0,0,Treep52,2022-12-30 22:51:15+00:00,Per Alex: Approximately 95% of adults and 98% ...,1,3425,3949,...,1,0,0,0.333333,2,0,M,1,3.633333,3.633333
1,1,1,1,1,DrGurdeepParhar,2022-12-30 21:10:00+00:00,Big Tobacco's divestment from Quebec's Medicag...,0,66607,48,...,0,0,0,0.000000,2,0,M,1,4.317857,4.317857
2,2,2,2,2,stopthegaslight,2022-12-30 18:49:59+00:00,@brianlilley COVID Vaccinations are the most u...,0,17,189,...,0,0,0,0.000000,3,0,M,1,1.575000,1.575000
3,3,3,3,3,Daisy381,2022-12-30 17:31:01+00:00,@MerlinofCanada @JustinTrudeau In Alberta it’s...,1,331,598,...,1,1,0,0.666667,4,0,M,1,3.583333,3.583333
4,4,4,4,4,RadarsScience,2022-12-30 16:20:06+00:00,Big Tobacco&amp;#039;s divestment from Quebec&...,0,29,45,...,0,0,0,0.000000,2,1,C,1,1.317857,1.317857


In [37]:
df_vaccine.to_csv('/content/drive/My Drive/Misinformation_Project/Bert_classified_impact_score_30K_vaccine_21_22.csv')